In [13]:
import tensorflow as tf
import pandas as pd
from tensorflow import keras

import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

2.11.0-dev20221010


In [14]:
import numpy as np
import pandas as pd

In [15]:
train = pd.read_csv(r'DATA/spam.csv')
train.head()

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [16]:
test = pd.read_csv(r'DATA/spamtest.csv')
test.head()

,Category,Message
0,ham,I'm so in love with you. I'm excited each day ...
1,spam,-PLS STOP bootydelious (32/F) is inviting you ...
2,spam,BangBabes Ur order is on the way. U SHOULD rec...
3,ham,I place all ur points on e cultures module alr...
4,spam,URGENT! We are trying to contact you. Last wee...


In [18]:
train['Category'].value_counts()

ham     4825
spam     747
Name: Category, dtype: int64

this is an imbalanced data set we have to use some techniques to tackle it

In [23]:
df_spam = train[train['Category']=='spam']
df_spam.shape

(747, 2)

In [25]:
df_ham = train[train['Category']=='ham']
df_ham.shape

(4825, 2)

Easiest way is downspling the dataset

In [26]:
df_ham_downsampled = df_ham.sample(df_spam.shape[0])
df_ham_downsampled.shape

(747, 2)

In [27]:
df_balanced = pd.concat([df_ham_downsampled, df_spam])
df_balanced.shape

(1494, 2)

In [29]:
df_balanced['Category']=df_balanced['Category'].apply(lambda x: 1 if x=='spam' else 0)
df_balanced.sample(5)

,Category,Message
4548,0,Hey mate. Spoke to the mag people. We‘re on. ...
1631,0,I am going to film 2day da. At 6pm. Sorry da.
2784,0,"Just arrived, see you in a couple days &lt;3"
3232,0,No plm i will come da. On the way.
4377,1,"If you don't, your prize will go to another cu..."


In [30]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df_balanced['Message'],df_balanced['Category'], test_size=0.1)

In [31]:
from sklearn.feature_extraction.text import CountVectorizer
v = CountVectorizer()
X_train_count = v.fit_transform(X_train.values)
X_train_count.toarray()[:2]

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [32]:
from sklearn.naive_bayes import MultinomialNB
model = MultinomialNB()
model.fit(X_train_count,y_train)

MultinomialNB()

In [47]:
def predict_message(pred_text):

  emails_count = v.transform([pred_text])
  prediction = model.predict(emails_count)

  p = prediction.tolist()

  if p[0] == 0:
    p1 = "ham"

  else:
    p1 = "spam"


  return  (pred_text,p1)

pred_text = "i'll bring it tomorrow. don't forget the milk."

prediction = predict_message(pred_text)
print(prediction)

("i'll bring it tomorrow. don't forget the milk.", 'ham')


In [50]:
test_messages = test["Message"].head(10)

for msg in test_messages:
    prediction = predict_message(msg)
    print(prediction)

("I'm so in love with you. I'm excited each day i spend with you. You make me so happy.", 'ham')
('-PLS STOP bootydelious (32/F) is inviting you to be her friend. Reply YES-434 or NO-434 See her: www.SMS.ac/u/bootydelious STOP? Send STOP FRND to 62468', 'spam')
('BangBabes Ur order is on the way. U SHOULD receive a Service Msg 2 download UR content. If U do not, GoTo wap. bangb. tv on UR mobile internet/service menu', 'spam')
('I place all ur points on e cultures module already.', 'ham')
('URGENT! We are trying to contact you. Last weekends draw shows that you have won a £900 prize GUARANTEED. Call 09061701939. Claim code S89. Valid 12hrs only', 'spam')
("Hi frnd, which is best way to avoid missunderstding wit our beloved one's?", 'ham')
('Great escape. I fancy the bridge but needs her lager. See you tomo', 'ham')
('Yes :)it completely in out of form:)clark also utter waste.', 'ham')
('Sir, I need AXIS BANK account no and bank address.', 'ham')
('Hmmm.. Thk sure got time to hop ard... 